# CoursewareHub環境の削除

構築した環境を削除します。

# パラメータの指定

## VCCアクセストークンの入力

VCノードを削除するためにVC Controller(VCC)のアクセストークンが必要となります。
次のセルを実行すると表示される入力枠にVCCのアクセストークンを入力してください。

> アクセストークン入力後に Enter キーを押すことで入力が完了します。

In [ ]:
from getpass import getpass
vcc_access_token = getpass()

入力されたアクセストークンが正しいことを、実際にVCCにアクセスして確認します。

In [ ]:
from common import logsetting
from vcpsdk.vcpsdk import VcpSDK

vcp = VcpSDK(vcc_access_token)

上のセルの実行結果がエラーとなり以下のようなメッセージが表示されている場合は、入力されたアクセストークンに誤りがあります。

```
2018-09-XX XX:XX:XX,XXX - ERROR - config vc failed: http_status(403)
2018-09-XX XX:XX:XX,XXX - ERROR - 2018/XX/XX XX:XX:XX UTC: VCPAuthException: xxxxxxx:token lookup is failed: permission denied
```

エラーになった場合はこの節のセルを全て `unfreeze` してから、もう一度アクセストークンの入力を行ってください。

## グループ名

CoursewareHubの構築環境となるVCPのUnitGroup名を指定してください。

> VCノードを起動した際に指定したものと同じ名前を指定してください。

In [ ]:
unit_group = 'CoursewareHub'

# 構築環境の削除

## VCノードの削除

起動したVCノードを削除します。

現在のUnitGroupの一覧を確認します。

In [ ]:
vcp.df_ugroups()

現在のVCノードの状態を確認します。

In [ ]:
ugroup = vcp.get_ugroup(unit_group)
ugroup.df_nodes()

まず worker の VCノードを削除します。

> workerのVCノードがない場合は何もしません。

In [ ]:
worker_unit = ugroup.get_unit('worker')
if worker_unit:
    worker_unit.delete_nodes(num_delete_nodes=len(worker_unit.find_nodes()))

マスターのVCノードと UnitGroup の削除を行います。

In [ ]:
ugroup.cleanup()

削除後の UnitGroupの一覧を確認します。

In [ ]:
vcp.df_ugroups()

## VCディスクの削除

NFS用のVCディスクを削除します。

> VCディスクを作成していない場合は、何もしません。

現在の状態を確認します。

In [ ]:
from IPython.display import display
ug_disk = vcp.get_ugroup(unit_group + '_disk')
if ug_disk:
    display(ug_disk.df_nodes())

VCディスクを削除します。

In [ ]:
if ug_disk:
    ug_disk.cleanup()

削除後のUnitGroupの一覧を確認します。

In [ ]:
vcp.df_ugroups()

# Ansible設定のクリア

削除した環境に対応するAnsibleの設定をクリアします。

## group_varsファイル

group_varsファイルをリネームします。

In [ ]:
!mv group_vars/{unit_group}.yml group_vars/{unit_group}.yml.bak

## インベントリ

インベントリから UnitGroup に対応するグループを削除します。

In [ ]:
import yaml
from pathlib import Path

inventory_path = Path('hosts')

!cp {str(inventory_path)} {str(inventory_path)}.bak

with inventory_path.open() as f:
    inventory = yaml.safe_load(f)
    
if unit_group in inventory['all']['children']:
    del(inventory['all']['children'][unit_group])

with inventory_path.open(mode='w') as f:
    f.write(yaml.safe_dump(inventory))
    
!cat {str(inventory_path)}